In [2]:
import numpy as np 
import pandas as pd
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore') 

sns.set()

In [2]:
dataset_char = pd.read_csv("A_Z Handwritten Data/A_Z Handwritten Data.csv").astype('float32')
# dataset_char = pd.read_csv("A_Z Handwritten Data/A_Z Handwritten Data.csv")

dataset_char.rename(columns={'0':'label'}, inplace=True)
#Splite data the X - Our data , and y - the prdict label
X_char = dataset_char.drop('label',axis = 1)
y_char = dataset_char['label']

In [3]:
dataset_digit = pd.read_csv("mnist/mnist_frame.csv").astype('float32')
# dataset_digit = pd.read_csv("mnist/mnist_frame.csv")

dataset_digit.rename(columns={'0':'class'}, inplace=True)
#Splite data the X - Our data , and y - the prdict label
X_digit = dataset_digit.drop('class',axis = 1)
y_digit = dataset_digit['class']

In [4]:
X_train_char, X_test_char, y_train_char, y_test_char = train_test_split(X_char,y_char)

# scale data
# standard_scaler = MinMaxScaler()
# standard_scaler.fit(X_train_char)

# Convert training labels from numeric to characters
y_train_chars = y_train_char+10

# Convert test labels from numeric to characters
y_test_chars = y_test_char+10
# print(y_train_chars)

# X_train_char = standard_scaler.transform(X_train_char)
# X_test_char = standard_scaler.transform(X_test_char)

In [5]:
X_train_digit, X_test_digit, y_train_digit, y_test_digit = X_digit[:60000],X_digit[60000:],y_digit[:60000],y_digit[60000:]
# scale data
# standard_scaler = MinMaxScaler()
# standard_scaler.fit(X_train_digit)

# X_train_digit = standard_scaler.transform(X_train_digit)
# X_test_digit = standard_scaler.transform(X_test_digit)

In [6]:
X_train_char = X_train_char / 255.0
X_test_char = X_test_char / 255.0
X_train_digit = X_train_digit / 255.0
X_test_digit = X_test_digit / 255.0


In [7]:
X_train=np.vstack((X_train_char,X_train_digit))
X_test=np.vstack((X_test_char,X_test_digit))
y_train=np.hstack((y_train_chars,y_train_digit))
y_test=np.hstack((y_test_chars,y_test_digit))
y=pd.Series(np.hstack((y_train,y_test)))

In [8]:
X_train.shape

(339337, 784)

In [9]:
train_indices = np.random.permutation(len(X_train))
train_features = X_train[train_indices]
train_labels = y_train[train_indices]

# Shuffle the test data
test_indices = np.random.permutation(len(X_test))
test_features = X_test[test_indices]
test_labels = y_test[test_indices]


In [10]:
print(train_features)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
train_features.shape
train_labels.shape

(339337,)

In [12]:
# x=pd.DataFrame(train_features)
# y=train_labels
# plt.figure(figsize=(7,7))
# idx=5651

# grid_data = x.iloc[idx].values.reshape(28,28)
# plt.imshow(grid_data,interpolation="none",cmap="gray")
# plt.show()

# print(y[idx])

In [13]:
# from sklearn.utils import shuffle
# x=pd.DataFrame(train_features)
# X_shuffle = shuffle(x)

# plt.figure(figsize = (12,10))
# row, colums = 4, 4
# for i in range(16):  
#     plt.subplot(colums, row, i+1)
#     plt.imshow(X_shuffle.iloc[i].values.reshape(28,28),interpolation='nearest', cmap='Greys')
# plt.show()

In [14]:
train_features = train_features.reshape(train_features.shape[0], 28, 28, 1).astype('float32')
test_features = test_features.reshape(test_features.shape[0], 28, 28, 1).astype('float32')

train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels)

In [15]:
cls = Sequential()
cls.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
cls.add(MaxPooling2D(pool_size=(2, 2)))
cls.add(Dropout(0.3))
cls.add(Flatten())
cls.add(Dense(128, activation='relu'))
cls.add(Dense(len(y.unique()), activation='softmax'))

cls.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = cls.fit(train_features, train_labels, validation_data=(test_features, test_labels), epochs=18, batch_size=200, verbose=2)
0
scores = cls.evaluate(test_features,test_labels, verbose=0)
print("CNN Score:",scores[1])

Epoch 1/18
1697/1697 - 94s - loss: 0.2983 - accuracy: 0.9176 - val_loss: 0.1247 - val_accuracy: 0.9660 - 94s/epoch - 55ms/step
Epoch 2/18
1697/1697 - 93s - loss: 0.1297 - accuracy: 0.9623 - val_loss: 0.0962 - val_accuracy: 0.9739 - 93s/epoch - 55ms/step
Epoch 3/18
1697/1697 - 90s - loss: 0.1008 - accuracy: 0.9703 - val_loss: 0.0766 - val_accuracy: 0.9793 - 90s/epoch - 53ms/step
Epoch 4/18
1697/1697 - 90s - loss: 0.0845 - accuracy: 0.9747 - val_loss: 0.0758 - val_accuracy: 0.9794 - 90s/epoch - 53ms/step
Epoch 5/18
1697/1697 - 90s - loss: 0.0735 - accuracy: 0.9777 - val_loss: 0.0649 - val_accuracy: 0.9822 - 90s/epoch - 53ms/step
Epoch 6/18
1697/1697 - 90s - loss: 0.0642 - accuracy: 0.9801 - val_loss: 0.0581 - val_accuracy: 0.9840 - 90s/epoch - 53ms/step
Epoch 7/18
1697/1697 - 89s - loss: 0.0564 - accuracy: 0.9820 - val_loss: 0.0565 - val_accuracy: 0.9846 - 89s/epoch - 53ms/step
Epoch 8/18
1697/1697 - 89s - loss: 0.0503 - accuracy: 0.9837 - val_loss: 0.0543 - val_accuracy: 0.9849 - 89s/ep

In [16]:
# Save the trained model
cls.save("digit_char_model.h5")

In [1]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("digit_char_model.h5")


In [3]:
# def predict(image):
#     image = image.reshape(-1, 28, 28, 1)
#     im_resize=image/255.0
#     prediction = model.predict(im_resize)[0]
#     return {str(i): float(prediction[i]) for i in range(len(prediction))}

def predict(image):
    image = image.reshape(-1, 28, 28, 1)
    im_resize=image/255.0
    prediction = model.predict(im_resize)[0]
    
    if np.argmax(prediction) < 10:
        label = str(np.argmax(prediction))
    else:
        label = chr(ord('A') + np.argmax(prediction) - 10)
    
    return label



In [46]:
# def predict(image):
#     image = image.reshape(-1, 28, 28, 1)
#     im_resize=image/255.0
#     prediction = cls.predict(im_resize)[0]
#     return {str(i): float(prediction[i]) for i in range(len(prediction))}

In [4]:
# iface = gr.Interface(fn=predict, inputs="sketchpad", outputs=gr.outputs.Label(num_top_classes=len(y.unique())))
iface = gr.Interface(predict,inputs='sketchpad',outputs='label')

In [5]:
iface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 34ms/step
Keyboard interruption in main thread... closing server.
